## Alpha Vantage - PE Ratio Example

In [4]:
import os # for referencing environment variables
from dotenv import load_dotenv # for loading environment variables 
import requests

In [5]:
load_dotenv() # load environment variables from .env file
alpha_vantage_api_key = os.getenv('ALPHA_VANTAGE_API_KEY') # Get your API key from https://www.alphavantage.co/support/#api-key

In [6]:
ticker = 'AAPL'
url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={alpha_vantage_api_key}'
r = requests.get(url)
data = r.json()
pe_ratio = float(data['PERatio'])
pe_ratio

35.85

### Other Web Search Tools

In [ ]:
from langchain_community.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchResults()
search_result = search.invoke(query_string)

In [ ]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.document_loaders import SeleniumURLLoader
import re
wrapper = DuckDuckGoSearchAPIWrapper(max_results=4)
url_search = DuckDuckGoSearchResults(api_wrapper=wrapper)
url_search_result = search.invoke(query_string)
pattern = r'https?://[^\s\]]+'
#match = re.search(pattern, url_search_result)
#first_link = match.group() if match else None
#loader = SeleniumURLLoader(urls=[first_link])
#data = loader.load()
#page_content = data[0].page_content
urls = re.findall(pattern, url_search_result)
page_content = ''
for url in urls:
    loader = SeleniumURLLoader(urls=[url])
    data = loader.load()
    page_content = data[0].page_content

    # Check if the page content is not empty
    if page_content:
        break  # Exit the loop if non-empty content is found


In [ ]:
# def merge_summaries(summary1_json: str, summary2_json: str) -> str:
#     '''
#     Merge two instances of PortersFiveForcesSummary, keeping unique forces and retaining summary1's version of duplicates.
#     '''
#     if summary1_json is None or summary1_json == '':
#         summary1_json = str(PortersFiveForcesSummary(forces_scores=[]).json())
#     summary1 = PortersFiveForcesSummary.parse_raw(summary1_json)
#     if summary2_json is None or summary2_json == '':
#         summary2_json = str(PortersFiveForcesSummary(forces_scores=[]).json())
#     summary2 = PortersFiveForcesSummary.parse_raw(summary2_json)

#     # Create a dictionary for summary1 forces using the original attribute names
#     force_dict: Dict[str, PortersForceScore] = {force.force: force for force in summary1.forces}
    
#     # Add unique forces from summary2, ignoring duplicates
#     for force in summary2.forces:
#         if force.force not in force_dict:
#             force_dict[force.force] = force
    
#     # Create a new summary with merged forces using the alias 'forces_scores'
#     merged_summary = PortersFiveForcesSummary(forces_scores=list(force_dict.values()))

#     merged_summary_json = merged_summary.json()

#     return merged_summary_json

In [4]:
import yfinance as yf
import pandas as pd

def get_stock_data(ticker: str, period: str = '5y'):
    stock_data = yf.download(ticker, period=period)
    closing_prices = stock_data['Close']
    return closing_prices

In [5]:
ticker = 'AAPL' 
closing_prices = get_stock_data(ticker)

[*********************100%%**********************]  1 of 1 completed


In [3]:
closing_prices

Date
2019-08-07     49.759998
2019-08-08     50.857498
2019-08-09     50.247501
2019-08-12     50.119999
2019-08-13     52.242500
                 ...    
2024-08-01    218.360001
2024-08-02    219.860001
2024-08-05    209.270004
2024-08-06    207.229996
2024-08-07    210.899994
Name: Close, Length: 1259, dtype: float64

In [18]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import timedelta

def get_stock_data(ticker: str, period: str = '5y', expected_return: float = 0.1):
    stock_data = yf.download(ticker, period=period)
    closing_prices = stock_data['Close']
    last_close_price = closing_prices.iloc[-1]
    future_dates = pd.date_range(start=closing_prices.index[-1] + timedelta(days=1), periods=365, freq='D')
    future_prices = last_close_price * (1 + expected_return * np.linspace(0, 1, 365))
    future_prices_df = pd.DataFrame({'Close': future_prices}, index=future_dates)
    stock_prices = {
        'labels': closing_prices.index.strftime('%Y-%m-%d').tolist() + future_dates.strftime('%Y-%m-%d').tolist(),
        'datasets': [
            {
                'label': 'History',
                'data': closing_prices.values.tolist() + [None] * 365,
                'borderColor': 'rgba(75, 192, 192, 1)',
                'backgroundColor': 'rgba(75, 192, 192, 0.2)',
                'fill': False,
            },
            {
                'label': 'Forecast',
                'data': [None] * len(closing_prices) + future_prices.tolist(),
                'borderColor': 'rgba(255, 99, 132, 1)',
                'backgroundColor': 'rgba(255, 99, 132, 0.2)',
                'fill': False,
            }
        ]
    }
    return stock_prices

In [19]:
ticker = 'AAPL'
expected_return = 0.1
chart_data = get_stock_data(ticker, expected_return=expected_return)

[*********************100%%**********************]  1 of 1 completed


In [20]:
type(chart_data)

dict

In [ ]:
def get_stock_data(state: State) -> State:
    ticker = state['ticker']
    expected_return = state['expected_return']
    period = '5y'
    stock_data = yf.download(ticker, period=period)
    closing_prices = stock_data['Close']
    last_close_price = closing_prices.iloc[-1]
    future_dates = pd.date_range(start=closing_prices.index[-1] + timedelta(days=1), periods=365, freq='D')
    future_prices = last_close_price * (1 + expected_return * np.linspace(0, 1, 365))
    future_prices_df = pd.DataFrame({'Close': future_prices}, index=future_dates)
    stock_prices = {
        'labels': closing_prices.index.strftime('%Y-%m-%d').tolist() + future_dates.strftime('%Y-%m-%d').tolist(),
        'datasets': [
            {
                'label': 'History',
                'data': closing_prices.values.tolist() + [None] * 365,
                'borderColor': 'rgba(75, 192, 192, 1)',
                'backgroundColor': 'rgba(75, 192, 192, 0.2)',
                'fill': False,
            },
            {
                'label': 'Forecast',
                'data': [None] * len(closing_prices) + future_prices.tolist(),
                'borderColor': 'rgba(255, 99, 132, 1)',
                'backgroundColor': 'rgba(255, 99, 132, 0.2)',
                'fill': False,
            }
        ]
    }
    state['stock_prices'] = stock_prices
    return state